In [1]:
%cd C:\Users\yukir\Documents\GitHub\Text_Mining\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA

C:\Users\yukir\Documents\GitHub\Text_Mining\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13292188212082574544
]
tf 2.5.3
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


C:\Users\yukir\Documents\Monicas_workspace\Korea_medicine\LDA


In [2]:
import openpyxl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv

from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

### 띄어쓰기

In [3]:
%cd C:\Users\yukir\Documents\Monicas_workspace\Derma

C:\Users\yukir\Documents\Monicas_workspace\Derma


In [4]:
reviews_df = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\[0430]repreprocessed.xlsx", index_col=0)
print(reviews_df.shape)
reviews_df.head()

(31294, 9)


,h,reviews,stars,review_date,do,si,ro,cleaned_reviews,reviews_stem
0.0,고운나라피부과의원,두피가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담해...,9.3,"3월 15일, 2022년",경남,김해시,내외중앙로,두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,"['두피', '가려움증', '원한', '병원', '진료', '두피', '현미경', ..."
1.0,고운나라피부과의원,내과,10,"12월 20일, 2021년",경남,김해시,내외중앙로,대상포진으로 진료받았어요 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉았...,"['대상포진', '진료', '주', '이상', '약', '주사', '진통', '대상..."
2.0,고운나라피부과의원,여드름 고민이 있어서 방문했습니다피부 전문의라서 증상이 나타난 이유에 대해서 설명을...,10,"8월 11일, 2021년",경남,김해시,내외중앙로,여드름 고민이 있어서 방문했습니다 피부전문의라서 증상이 나타난 이유에 대해서 설명을...,"['여드름', '고민', '방문', '피부', '전문의', '증상', '이유', '..."
3.0,고운나라피부과의원,문신제거문신한곳 일부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 천차...,10,"6월 21일, 2021년",경남,김해시,내외중앙로,문신제거 문신한 곳일 부분만 지우고 싶어서 여러 군데 알아보고 갔어요 워낙 가격이 ...,"['문신', '제거', '문신', '곳일', '부분', '여러', '군데', '워낙..."
4.0,고운나라피부과의원,원장님이 설명도 잘해주시고 병원도 엄청크고~(김해에서 유명한곳은 이유가있네요)!! ...,9.5,"6월 7일, 2021년",경남,김해시,내외중앙로,원장님이설명도잘해주시고병원도엄청크고김해에서유명한곳은이유가있네요직원들도전부친절해서넘맘...,"['원장', '설명', '병원', '김해', '곳', '유가', '직원', '전부'..."


In [5]:
reviews_df.shape

(31294, 9)

In [10]:
data = reviews_df.copy()
# data = data[20000:]
print(data.shape)

(31294, 9)


In [7]:
def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = True)
    # print(pos)
    for j in pos:
        if j.split('/')[1] == 'Noun':
            j = j.split('/')[0]
            morphs_list.append(j)
                    
        elif j.split('/')[1] =='Adjective':
            k = okt.morphs(j,  stem= True)
            k = k[0]
            morphs_list.append(k)
        elif j.split('/')[1] =='Verb':
            v = okt.morphs(j,  stem= True)
            v = v[0]
            morphs_list.append(v)

    for i in morphs_list:
        if len(i) != 1:
            one_words.append(i)
    

    for i in one_words:
        if i not in stopwords_kor:
            result.append(i)

    return result

In [18]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 
    
def get_nouns(tokenizer, sentence): 
    """ 단어의 길이가 2이상인 일반명사(NNG),
     고유명사(NNP), 외국어(SL)만을 반환한다. 
     :param tokenizer: 
     :param sentence: :return: """ 
    tagged = tokenizer.nouns(sentence)
    nouns = [s for s in tagged if len(s)>1] 
     
    return nouns 
     
     
def tokenize(df): 
    tokenizer = okt
    processed_data = [] 
    for sent in tqdm(df['cleaned_reviews']):
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
        
    return processed_data
    
    
def save_processed_data(processed_data): 
    """ 토큰 분리한 데이터를 csv로 저장 :param processed_data: :return: """ 
    
    with open('[0430]tf_idf_token.csv', 'w', newline='', encoding='utf-8') as f: 
        writer = csv.writer(f) 
        for data in processed_data:
             writer.writerow(data)

### TF_IDF

In [19]:
print(len(data))
data = data.dropna()
print(len(data))
data = data.drop_duplicates('reviews')
print(len(data))

29336
29336
29336


In [20]:
for sent in tqdm(data['reviews']):
    try:
        sentence = clean_text(sent.replace('\n', '').strip())
        # print(sentence)
    except:
        # print(sent)
        pass

100%|██████████| 29336/29336 [00:00<00:00, 185670.90it/s]


In [21]:
## 오래걸리니까 조심

if __name__ == '__main__':
     # df = pd.read_csv('negative.csv', index_col = 0)
     processed_data = tokenize(data) # description 부분을 토크나이징 한다. 
     
     # 토큰 분리한 데이터를 저장 
     save_processed_data(processed_data)


100%|██████████| 29336/29336 [10:42<00:00, 45.64it/s] 


In [22]:
detokenized_doc = []
for i in range(len(processed_data)):
    t = ' '.join(processed_data[i])
    detokenized_doc.append(t)

data['detokenized_doc'] = detokenized_doc # 다시 text['headline_text']에 재저장

data['detokenized_doc'][:5]

0.0      두피 가려움증 원한 병원 진료 두피 현미경 사진 무료 상담 두피 샴푸 하나 샴푸 방법
1.0           대상포진 진료 이상 주사 진통 대상포진 통증 후유증 남아 부위 계속 예방접종
2.0    여드름 고민 방문 피부 전문의 증상 이유 대해 설명 지인 추천 의사 선생님 진료 상...
3.0    문신 제거 문신 곳일 부분 여러 군데 워낙 가격 천차만별 치료 합리 가격 부분 레이...
4.0                     원장 설명 병원 김해 유가 직원 전부 처방 하루 피부 아주
5.0         지루 피부염 병원 돋보기 유심 피부 상태 확인 연고 의약품 처방 환자 법도 대해
Name: detokenized_doc, dtype: object

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=stopwords_kor,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[가-힣0-9]{1,}',  # num chars > 3
                             max_features=2000,             # max number of uniq words
                            ) 
data_vectorized = vectorizer.fit_transform(data['detokenized_doc'])
data_vectorized.shape # TF-IDF 행렬의 크기 확인

(29336, 2000)

In [24]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
dist = np.sum(data_vectorized, axis=0)
    
df_freq = pd.DataFrame(dist, columns=vocab)
print(df_freq.shape)

2000
(1, 2000)


In [25]:
df_freq.T.sort_values(by=0, ascending=False).head(100)
df_freq_T = df_freq.T.reset_index()
df_freq_T.columns = ["words", "freq"]
df_freq_T.tail()

,words,freq
1995,희망,7.601364
1996,희재,5.664223
1997,히스타민,5.193308
1998,힐러,16.061285
1999,힐링,4.943017


In [26]:
print(df_freq_T.shape)
df_use = df_freq_T.drop_duplicates()
print(df_use.shape)

(2000, 2)
(2000, 2)


In [27]:
df_use = df_use.sort_values(by="freq", ascending=False)
df_use.head(100)

,words,freq
1691,치료,1672.548118
834,선생님,1593.273588
1284,의사,1375.643738
840,설명,1342.504621
1633,처방,1260.180963
...,...,...
244,기억,197.102634
1572,지인,190.201512
468,마취,189.818192
796,상처,184.590742


In [28]:
df_use.to_excel('[0430]derma_TFIDF_to_SERVQUAL_v5.xlsx')

In [29]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.6163399918189255 %


In [30]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=1337,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_top=lda_model.fit_transform(data_vectorized)

print(lda_model.components_)
print(lda_model.components_.shape) 

[[3.14923630e+01 7.93790702e+00 2.06109883e-01 ... 2.05666407e-01
  2.01030725e-01 5.28308387e+00]
 [7.69677815e+00 4.78707171e+00 3.76807783e+01 ... 2.01125571e-01
  2.01041872e-01 2.01183914e-01]
 [2.55959537e+01 9.26392384e+01 2.06801516e-01 ... 5.64927692e+00
  2.00806058e-01 2.01846265e-01]
 [1.15787276e+02 2.71437903e+01 2.08404377e-01 ... 2.10865646e-01
  2.00260535e-01 2.02158835e-01]
 [3.31688533e+01 5.36873846e+02 1.91086823e+01 ... 2.02775954e-01
  1.55837048e+01 2.11433587e-01]]
(5, 2000)


In [31]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data['reviews'].tolist()))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.160000,0.240000,0.050000,0.490000,0.060000,3
Doc1,0.070000,0.080000,0.070000,0.700000,0.070000,3
Doc2,0.040000,0.100000,0.070000,0.430000,0.360000,3
Doc3,0.060000,0.070000,0.650000,0.060000,0.160000,2
Doc4,0.320000,0.480000,0.070000,0.070000,0.070000,1
Doc5,0.050000,0.380000,0.050000,0.460000,0.050000,3
Doc6,0.070000,0.070000,0.720000,0.070000,0.070000,2
Doc7,0.540000,0.130000,0.230000,0.050000,0.050000,0
Doc8,0.790000,0.050000,0.050000,0.050000,0.050000,0
Doc9,0.770000,0.060000,0.060000,0.060000,0.060000,0


In [32]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('두드러기', 355.48), ('알레르기', 273.8), ('처방', 272.61), ('선생님', 272.26), ('의사', 214.01), ('설명', 211.07), ('약도', 153.6), ('리뷰', 150.71), ('간호사', 143.46), ('복용', 142.14)]
Topic 2: [('필러', 245.14), ('치료', 230.45), ('제거', 228.73), ('사마귀', 228.16), ('탈모', 176.23), ('수술', 169.64), ('냉동', 142.18), ('티눈', 132.53), ('답변', 116.78), ('친구', 101.73)]
Topic 3: [('여드름', 1193.14), ('염증', 714.44), ('주사', 643.93), ('치료', 512.3), ('압출', 432.63), ('약물', 329.34), ('무좀', 300.84), ('선생님', 287.63), ('의사', 250.65), ('처방', 246.46)]
Topic 4: [('처방', 702.36), ('선생님', 508.57), ('설명', 469.88), ('의사', 439.08), ('치료', 428.07), ('증상', 418.12), ('연고', 360.84), ('알레르기', 303.68), ('원장', 288.08), ('대기', 285.0)]
Topic 5: [('시술', 982.15), ('레이저', 785.5), ('보톡스', 679.01), ('상담', 620.66), ('가격', 536.87), ('제거', 519.9), ('예약', 483.79), ('직원', 452.17), ('선생님', 444.3), ('효과', 413.5)]


In [33]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
C:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\sklearn\manifold\_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  31.515735
3       51.163364 -159.968018       2        1  24.810313
2      -73.144554   21.523815       3        1  16.889279
0      -66.410133 -110.606438       4        1  14.710187
1       54.417324   12.443444       5        1  12.074486, topic_info=     Term         Freq        Total Category  logprob  loglift
1121  여드름  1247.000000  1247.000000  Default  30.0000  30.0000
1152   염증   937.000000   937.000000  Default  29.0000  29.0000
1062   압출   452.000000   452.000000  Default  28.0000  28.0000
1473   제거   732.000000   732.000000  Default  27.0000  27.0000
1532   주사  1107.000000  1107.000000  Default  26.0000  26.0000
...   ...          ...          ...      ...      ...      ...
1657   체크    69.298830   114.150159   Topic5  -5.0697   1.6150
1361   입술    67.639696   128.162262   Topic5  -5.0939   1.4750
834   선생님    88.426102  1546.146993   Topic5  -4.8260  -0.7473
1284   의사    74.051159  1329.201649   Topic5  -5.0034  -0.7735
1241   원장    66.579707   956.035293   Topic5  -5.1097  -0.5503

[295 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1         1  0.778081   가격
1         2  0.040866   가격
1         3  0.158559   가격
1         4  0.013077   가격
1         5  0.009808   가격
...     ...       ...  ...
1975      4  0.113976   효과
1975      5  0.018133   효과
1990      1  0.666278   흉터
1990      2  0.143435   흉터
1990      3  0.185077   흉터

[374 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 1, 2])